# Group By: split-apply-combine
+ **Splitting**: split the data into groups based on some criteria
+ **Applying**: apply a function to each group independently
  + Aggregation: group summary statistic (sum, means, group sizes/counts...)
  + Transformation: group-specific computations (standardizing data, filling NAs...)
  + Filtration: discard groups, group-wise computation
  + Above combination
+ **Combining**: combine the reuslts into a data structure

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'A':['c', 'b', 'a', 'b', 'b', 'c'],
                   'B':['one', 'two', 'two', 'three','three', 'three'],
                   'C': np.random.randint(3, size=6),
                   'D': np.random.randint(4, size=6)})

In [3]:
df

,A,B,C,D
0,c,one,0,3
1,b,two,2,1
2,a,two,2,0
3,b,three,2,3
4,b,three,0,2
5,c,three,1,3


## Splitting

In [4]:
# group by A, B or A&B
df.groupby('A')
df.groupby('B')
df.groupby(['A', 'B'])

In [5]:
# group keys sorting vs without sorting
print df.groupby('A').sum()
print ' '
print df.groupby('A', sort=False).sum()

   C  D
A      
a  2  0
b  4  6
c  1  6
 
   C  D
A      
c  1  6
b  4  6
a  2  0


In [6]:
# group selection
df.groupby(['A']).get_group('b')

,A,B,C,D
1,b,two,2,1
3,b,three,2,3
4,b,three,0,2


In [7]:
# group dict format
df.groupby('A').groups

{'a': Int64Index([2], dtype='int64'),
 'b': Int64Index([1, 3, 4], dtype='int64'),
 'c': Int64Index([0, 5], dtype='int64')}

In [8]:
# group interating
group = df.groupby('A')
for key, value in group:
    print key
    print value

a
   A    B  C  D
2  a  two  2  0
b
   A      B  C  D
1  b    two  2  1
3  b  three  2  3
4  b  three  0  2
c
   A      B  C  D
0  c    one  0  3
5  c  three  1  3


## Applying
### 1. Aggregation

In [9]:
df

,A,B,C,D
0,c,one,0,3
1,b,two,2,1
2,a,two,2,0
3,b,three,2,3
4,b,three,0,2
5,c,three,1,3


In [10]:
# aggregate sum, only apply to numerics
df.groupby('A').sum() # mean(), std()

,C,D
A,,
a,2,0
b,4,6
c,1,6


In [11]:
# group size
df.groupby('A').size()

A
a    1
b    3
c    2
dtype: int64

In [12]:
# group stats
df.groupby('A').describe()

C                                                    D                 \
  count      mean       std  min   25%  50%   75%  max count mean  std  min   
A                                                                             
a   1.0  2.000000       NaN  2.0  2.00  2.0  2.00  2.0   1.0  0.0  NaN  0.0   
b   3.0  1.333333  1.154701  0.0  1.00  2.0  2.00  2.0   3.0  2.0  1.0  1.0   
c   2.0  0.500000  0.707107  0.0  0.25  0.5  0.75  1.0   2.0  3.0  0.0  3.0   

                       
   25%  50%  75%  max  
A                      
a  0.0  0.0  0.0  0.0  
b  1.5  2.0  2.5  3.0  
c  3.0  3.0  3.0  3.0

In [13]:
# apply multiple functions
df.groupby('A').agg([np.sum, np.mean, np.std])

C                       D          
  sum      mean       std sum mean  std
A                                      
a   2  2.000000       NaN   0    0  NaN
b   4  1.333333  1.154701   6    2  1.0
c   1  0.500000  0.707107   6    3  0.0

### 2. Transformation

In [14]:
df

,A,B,C,D
0,c,one,0,3
1,b,two,2,1
2,a,two,2,0
3,b,three,2,3
4,b,three,0,2
5,c,three,1,3


In [15]:
# standardize within each group
zscore = lambda x: (x - x.mean()) / x.std()
df.groupby('A').transform(zscore)

,C,D
0,-0.707107,NaN
1,0.577350,-1.0
2,NaN,NaN
3,0.577350,1.0
4,-1.154701,0.0
5,0.707107,NaN


In [16]:
# column C + 1
df.groupby('A').C.transform(lambda x: x+1)

0    1
1    3
2    3
3    3
4    1
5    2
Name: C, dtype: int32

### 3. Filtration

In [17]:
df

,A,B,C,D
0,c,one,0,3
1,b,two,2,1
2,a,two,2,0
3,b,three,2,3
4,b,three,0,2
5,c,three,1,3


In [18]:
# filter the groups that group sum < 2 
df.groupby('A').D.filter(lambda x: x.sum() < 2)

2    0
Name: D, dtype: int32